# Schedule Generator

This code creates Lustre files with different numbers of jobs needing to be scheduled and runs Kind2 on them.  Jobs are created using two parameters:
  * **cost_range**: A pair of integers representing the lower and upper bounds on running time
  * **interval**: An integer representing the interval after which the jobs repeat
  
Easy scheduling tasks involve large intervals, small numbers of jobs, and small cost ranges.  As the intervals gets smaller, the number of jobs gets bigger, and the cost range gets larger it becomes more difficult to randomly get a workable schedule.

In [ ]:
import random
import os

In [ ]:
def make_job(cost_range, interval):
    return 'schedule(%d, %d, %d)' % \
           (random.randrange(interval),
            random.randrange(*cost_range),
            interval)

In [ ]:
def make_jobs(njobs, cost_range, interval):
    jobs = []
    for i in range(njobs):
        jobs.append('    j%d_status = %s;' % (i, make_job(cost_range, interval)))
    return jobs

In [ ]:
def make_lustre(njobs, cost_range, interval):
    """
    Create a Luster file that defines a number of jobs with start times, run times,
    and repeat intervals, and asserts the property that nome of the jobs overlap.
    
    Arguments:
      njob       - Number of jobs
      cost_range - Tuple of the form (shortest run time, longest run time)
      interval   - Repeat interval
      
    Returns: A string that represents the problem in Luster format
    """
    
    output = open('schedule.template').read()
    
    # Define variables
    output += 'node processor() returns('
    for i in range(njobs):
        output += 'j%d_status:bool;' % i
    output += ');\n'
    output += 'var no_conflict : bool;\n'

    # Create the jobs
    jobs = make_jobs(njobs, cost_range, interval)
    output += 'let\n'
    output += '\n'.join(jobs) + '\n'
    
    # Define the property to test
    output += '    no_conflict = not ('
    tests = []
    for i in range(njobs):
        for j in range(i + 1, njobs):
            tests.append('(j%d_status and j%d_status)' % (i, j))
    output += ' or '.join(tests)
    output += ');\n'
    output += '    --%PROPERTY no_conflict;\n'
    output += '\ntel'
    
    return output

### Example output

Note that the propery is more likely to fail with larger numbers of jobs, shorter repeat intervals, and longer running times (costs).

In [ ]:
print(make_lustre(3, (2, 3), 20))

In [ ]:
for i in range(10):
    print('Iteration %d' % i)
    lustre = make_lustre(3, (2, 3), 25)
    with open('test.lus', 'w') as file:
        file.write(lustre)
        
    #
    # Replace this line with whatever is needed to run Kind2 on an input file
    # named test.lus and dump the output of Kind2 to a file named OUT
    #
    
    !docker run -v /Users/oates/Desktop/mydocuments/funding/current/oyster/Kind2:/lus kind2/kind2 /lus/test.lus > OUT
        
    for line in open('OUT').readlines():
        if 'valid' in line:
            print(line.strip())

    os.rename('test.lus', 'test_%02d.lus' % i)
    os.rename('OUT', 'kind2_%02d.txt' % i)